In [294]:
import pandas as pd
import glob
import pickle

In [295]:
# importar y concatenar todos los datasets anuales
path = './datasets'
all_files = glob.glob(path + "/*.csv")

defunc = []
for file in all_files:
    df = pd.read_csv(file, encoding = "latin1")
    defunc.append(df)
    
defunciones = pd.concat(defunc, axis = 0, ignore_index= True)

In [296]:
# CHECKEAR DATAFRAME
defunciones.head()

,PROVRES,SEXO,CAUSA,MAT,GRUPEDAD,CUENTA
0,50,2,R99,,11_50 a 54,10
1,14,1,F10,,12_55 a 59,1
2,62,1,N05,,17_80 y más,1
3,74,2,C26,,17_80 y más,3
4,6,1,G30,,13_60 a 64,3


In [297]:
# CHECEKAR SI HAY NULLS
defunciones.isna().sum()

PROVRES          0
SEXO             0
CAUSA            0
MAT         565602
GRUPEDAD         0
CUENTA           0
dtype: int64

In [298]:
# ELIMINAR COLUMNA MAT PORQUE NO LA VOY A USAR
defunciones.drop('MAT', axis= 1, inplace= True)

In [299]:
# CHECKEAR CUÁNTAS ENTRADAS HAY DE SEXO "SIN ESPECIFICAR"
#defunciones.SEXO.value_counts()
# ELIMINARLAS
defunciones = defunciones.loc[defunciones['SEXO'] != 9]

In [59]:
# REEMPLAZAR CÓDIGO DE SEXO POR VALOR
TODO

NameError: name 'TODO' is not defined

In [300]:
# SACAR EL PREFIJO DE CADA GRUPO PARA QUE QUEDE SÓLO EL NOMBRE DEL GRUPO ETARIO
defunciones.GRUPEDAD = defunciones.GRUPEDAD.apply(lambda x: x.split('_')[1])

In [301]:
# CHECKEAR distribución de datos por grupos etarios
#defunciones.GRUPEDAD.value_counts()
# ELIMINAR REGISTROS CON GRUPO ETARIO SIN ESPECIFICAR
defunciones = defunciones.loc[defunciones['GRUPEDAD'] != 'Sin especificar']

In [290]:
'''
# CAMBIAR LOS RANGOS EXPLICITOS EN FORMATO STRING POR RANGOS NUMÉRICOS
for a, b in defunciones.GRUPEDAD.iteritems():
    if b == '80 y más':
        defunciones.at[a, 'GRUPEDAD'] = list(range(80,115))
    elif b == 'Menor de 1 año':
        defunciones.at[a, 'GRUPEDAD'] = 0
    else:
        defunciones.at[a, 'GRUPEDAD'] = list(range(int(b.split(' a ')[0]), int(b.split(' a ')[1]) + 1))
'''

In [302]:
# IMPORTAR REFERENCIAS DE CÓDIGOS PARA CADA COLUMNA
provincias = pd.read_excel('./datasets/DescDef1 (1).xlsx', sheet_name= 'PROVRES')
causa = pd.read_excel('./datasets/DescDef1 (1).xlsx', sheet_name= 'CODMUER')

In [303]:
# CHECKEAR DF PROVINCIAS Y CAMBIAR NOMBRES DE COLUMNAS
#provincias.head()
provincias.columns = ['PROVRES', 'PROVINCIA']

In [304]:
# CHECKEAR DF CAUSA Y CAMBIAR NOMBRES DE COLUMNAS
#causa.head()
causa.columns = ['CAUSA', 'DESCRIPCION']

In [354]:
# guardar causa como csv
causa.to_csv('./causa.csv', index= False)

In [305]:
# CHECKEAR CANTIDAD DE PROVINCIA = OTRO PAIS Y LUGAR NO ESPECIFICADO (CODS 98 Y 99)
# defunciones['PROVRES'].value_counts()
# ELIMINAR ESOS REGISTROS
defunciones = defunciones.loc[(defunciones['PROVRES'] != 99) & (defunciones['PROVRES'] != 98)]

In [306]:
# Generar diccionarios de códigos de provincias y causas a partir del df 
prov_dict = provincias.set_index('PROVRES')['PROVINCIA'].to_dict()
def_dict = causa.set_index('CAUSA')['DESCRIPCION'].to_dict()

In [307]:
# REEMPLAZAR LOS VALORES DE LA COLUMNA MAPEANDO A LOS VALORES DEL DICCIONARIO RECIÉN CREADO
#OPCION --> defunciones.replace({"PROVRES": prueba})
defunciones['PROVRES'] = defunciones["PROVRES"].map(prov_dict)

In [308]:
# AGRUPAR DATOS POR PROVINCIA, SEXO, GRUPO ETARIO Y CAUSA, sumar cuentas
agrupado = defunciones.groupby(['PROVRES', 'SEXO', 'GRUPEDAD', 'CAUSA']).sum()

In [309]:
# RESETEAR LOS INDICES PARA TENER UNA FILA POR CADA COMBINACIÓN DE CONDICIONES
agrupado.reset_index(inplace= True)

In [340]:
# GENERAR UNA TABLA PIVOTE A PARTIR DE LOS DATOS AGRUPADOS PARA QUE LAS COLUMNAS PASEN A SER LAS CAUSAS Y LOS VALORES LA CUENTA
por_causa = pd.pivot_table(agrupado, index= ['PROVRES', 'SEXO', 'GRUPEDAD'], values= 'CUENTA', columns= 'CAUSA')

In [341]:
# RESETEAR LOS INDICES PARA TENER UNA FILA POR CADA COMBINACIÓN DE CONDICIONES
por_causa.reset_index(inplace= True)

In [344]:
por_causa

CAUSA,PROVRES,SEXO,GRUPEDAD,A01,A02,A03,A04,A05,A06,A07,...,Y80,Y81,Y82,Y83,Y84,Y85,Y86,Y87,Y88,Y89
0,Buenos Aires,1,"(1, 2, 3, 4, 5, 6, 7, 8, 9)",NaN,NaN,NaN,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,20.0,NaN,NaN,NaN,NaN,NaN,NaN
1,Buenos Aires,1,"(10, 11, 12, 13, 14)",NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN
2,Buenos Aires,1,"(15, 16, 17, 18, 19)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8.0,1.0,NaN,NaN,NaN,NaN,NaN
3,Buenos Aires,1,"(20, 21, 22, 23, 24)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN
4,Buenos Aires,1,"(25, 26, 27, 28, 29)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,10.0,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
811,Tucumán,2,"(65, 66, 67, 68, 69)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,28.0,NaN,NaN,NaN,NaN,NaN,NaN
812,Tucumán,2,"(70, 71, 72, 73, 74)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,33.0,NaN,NaN,NaN,NaN,NaN,NaN
813,Tucumán,2,"(75, 76, 77, 78, 79)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,24.0,NaN,NaN,1.0,NaN,NaN,NaN
814,Tucumán,2,"(80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 9...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,87.0,1.0,NaN,NaN,NaN,NaN,NaN


In [343]:
# CAMBIAR LOS RANGOS EXPLICITOS EN FORMATO STRING POR RANGOS NUMÉRICOS
for a, b in por_causa.GRUPEDAD.iteritems():
    if b == '80 y más':
        por_causa.at[a, 'GRUPEDAD'] = range(80,115)
    elif b == 'Menor de 1 año':
        por_causa.at[a, 'GRUPEDAD'] = range(0,1)
    else:
        por_causa.at[a, 'GRUPEDAD'] = range(int(b.split(' a ')[0]), int(b.split(' a ')[1]) + 1)

In [346]:
por_causa.to_pickle('por_causa.pkl')

In [278]:
por_causa.to_csv('por_causa.csv', index= False)

--------------------

### HACER ESTO DIRECTAMENTE EN EL SCRIPT

In [348]:
a = test2.loc[(test2['PROVRES'] == 'Buenos Aires') & (test2['SEXO'] == 1) & (test2['GRUPEDAD'] == '1 a 9')]

In [349]:
a.dropna(axis= 1, inplace= True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [350]:
a

CAUSA,PROVRES,SEXO,GRUPEDAD,A04,A05,A09,A16,A17,A18,A19,...,Y19,Y20,Y21,Y24,Y26,Y28,Y33,Y34,Y59,Y83
0,Buenos Aires,1,1 a 9,1.0,1.0,21.0,4.0,3.0,1.0,2.0,...,2.0,17.0,81.0,18.0,43.0,4.0,10.0,145.0,1.0,20.0


In [161]:
a.drop(['PROVRES', 'SEXO', 'GRUPEDAD'], axis= 1)

CAUSA,A04,A05,A09,A16,A17,A18,A19,A32,A37,A39,...,Y19,Y20,Y21,Y24,Y26,Y28,Y33,Y34,Y59,Y83
0,1.0,1.0,21.0,4.0,3.0,1.0,2.0,2.0,1.0,20.0,...,2.0,17.0,81.0,18.0,43.0,4.0,10.0,145.0,1.0,20.0


In [351]:
b = a.drop(['PROVRES', 'SEXO', 'GRUPEDAD'], axis= 1).sort_values(by = 0, axis= 1, ascending= False)

In [352]:
b

CAUSA,X59,R99,J18,A41,Q24,W74,V49,Y34,C91,G93,...,I74,I77,I87,I89,I99,J13,A05,J35,J43,J14
0,269.0,242.0,240.0,229.0,210.0,190.0,149.0,145.0,144.0,139.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [181]:
def_dict[b.columns[6]]

'Ocupante de automóvil lesionado en otros accidentes de transporte y en los no especificados'